In [1]:
import random
import numpy as np
import math

In [43]:
def gen_problem_input(n, m):
    #Generate input for the airport problem
    #n airports, m flights
    
    input_s = ""
    
    #first line is n, m
    input_s += str(n) + " " + str(m) + "\n"
    #second line is p1 - pn
    for i in range(n):
        input_s += str(random.randint(0, 1000000)) + " "
    input_s = input_s.rstrip()
    input_s += "\n"
    #next n lines are travel times to each airport from each airport
    for i in range(n):
        for j in range(n):
            if i == j:
                input_s += "0 "
            else:
                input_s += str(random.randint(0, 1000000)) + " "
        input_s = input_s.rstrip()
        input_s += "\n"
    #the next m lines are flights, with (origin, destination, time) format
    for i in range(m):
        o, d = random.sample(range(1, n+1), 2)
        input_s += str(o) + " "
        input_s += str(d) + " "
        input_s += str(random.randint(1, 1000000)) + "\n"
        
    return input_s

In [94]:
def airport_input_to_dicts(airport_input):
    problems = []
    lines = airport_input.splitlines()
    lines = [line.split(" ") for line in lines]
    
    while lines:
        problem = {}
        problem['n'] = int(lines[0][0])
        problem['m'] = int(lines[0][1])
        problem["idle_times"] = lines[1]
        problem["airport_graph"] = lines[2:2 + problem['n']]
        problem["flights"] = sorted(lines[2 + problem['n']:2 + problem['n'] + problem['m']], key = lambda flight: flight[2])
        
        lines = lines[2 + problem['n'] + problem['m']:]
        problems.append(problem)
    return problems

def minimum_planes(problem_input):
    
    problems = airport_input_to_dicts(problem_input)
    solutions = []
    
    for problem in problems:
        
        planes = []
        m = problem['m']
        n = problem['n']
        flights = problem["flights"]
        graph = problem["airport_graph"]
        idle_times = problem["idle_times"]
        
        for i in range(len(graph)):
            for j in range(len(graph[i])):
                if i != j:
                    graph[i][j] = int(graph[i][j]) + int(idle_times[i])
                else:
                    graph[i][j] = 0
                    
        #find the shortest paths to each node in the graph from
        #each origin and destination
        shortest_path_time = get_shortest_paths(graph, flights)

        #iterate throught the flights in time order
        time = 0
        while flights:
            
            next_flight = flights.pop(0)
            #move time up
            time = int(next_flight[2])
            #see if there's a plane that can take the flight
            available = None
            for p in planes:
                #if the plane can get from where it is to the airport the flight leaves from in time
                #then that plane can carry the flight
                if time - p["time"] >= shortest_path_time[(p["location"], next_flight[0])]:
                    available = p
                    break;
                    
            if available:
                available["time"] = time + shortest_path_time[(next_flight(0), next_flight(1))]
                available["location"] = next_flight(1)
            else:
                planes.append({"time": time + shortest_path_time[(next_flight(0), next_flight(1))],
                              "location": next_flight(1)})
                
        solutions.append(len(planes))
        
    return solutions
        

In [43]:
def brute_force_time(graph_size, origin_destination_set):
    #assumes 10 cycles per path, 4GHz processor
    o_g_perm = origin_destination_set * (origin_destination_set - 1)
    intermediary_path_perm = sum([math.factorial(graph_size - 2)/math.factorial(graph_size - 2 - i) for i in range(graph_size - 1)])
    paths = o_g_perm * intermediary_path_perm
    cycles_per_path = 10
    cycle_time = 1/(4 * math.pow(10, 9))
    seconds_per_year = 60 * 60 * 24 * 365
    return paths * cycles_per_path * cycle_time / seconds_per_year

In [64]:
brute_force_time(16, 10) * 365 * 24

222.16421700475001

In [95]:
minimum_planes(gen_problem_input(2,2))

[['0', '376879'], ['962420', '0']]
{'2', '1'}
['602281', '558810']
[['0', '376879'], ['962420', '0']]
[[0, 979160], [1521230, 0]]
